In [26]:
import urllib
from bs4 import BeautifulSoup
import pandas as pd
import csv
from random import randint
from time import sleep
import lxml
import re

def getInjuries(team, year):
    """
    Scrapes one page of injury information
    """

    # Scrape roster information
    content = urllib.urlopen("http://www.pro-football-reference.com/teams/" + team + "/" + year + "_roster.htm")
    s = content.read()
    soup = BeautifulSoup(s, 'lxml')

    table = soup.find_all('table')[1]
    roster = pd.read_html(str(table))[0]
    
    new_columns = roster.columns.values
    new_columns[1] = 'name'
    roster.columns = new_columns
    
    roster['name'] = [re.sub('[*]|[+]', '', name) for name in roster['name']]
    roster[roster['name'] != "Team Total"]
    roster['team'] = team
    roster['year'] = year
    
    roster = roster.drop(roster.index[-1]) # Get rid of "team total"
    
    roster = roster.loc[:,['team','year','name','G','Yrs','Age']]
    
    return roster

In [27]:
## Get all pages to scrape
teams = ["crd", "atl", "rav", "buf","car","chi","cin","cle","dal","den","det","gnb","htx","clt","jax","kan","mia","min",
           "nwe","nor","nyg","nyj","rai","phi","pit","sdg","sfo","sea","ram","tam","oti","was"]
years = ['2009','2010','2011','2012','2013','2014']
team_years = [(x,y) for x in teams for y in years]

## Scrape each of the pages
output = pd.DataFrame()
for a,b in team_years:
    print a, b
    sleep(randint(1,3))
    sleep(randint(1,3))    
    output = output.append(getInjuries(a,b))

output['year'] = output['year'].astype('int')

crd 2009
crd 2010
crd 2011
crd 2012
crd 2013
crd 2014
atl 2009
atl 2010
atl 2011
atl 2012
atl 2013
atl 2014
rav 2009
rav 2010
rav 2011
rav 2012
rav 2013
rav 2014
buf 2009
buf 2010
buf 2011
buf 2012
buf 2013
buf 2014
car 2009
car 2010
car 2011
car 2012
car 2013
car 2014
chi 2009
chi 2010
chi 2011
chi 2012
chi 2013
chi 2014
cin 2009
cin 2010
cin 2011
cin 2012
cin 2013
cin 2014
cle 2009
cle 2010
cle 2011
cle 2012
cle 2013
cle 2014
dal 2009
dal 2010
dal 2011
dal 2012
dal 2013
dal 2014
den 2009
den 2010
den 2011
den 2012
den 2013
den 2014
det 2009
det 2010
det 2011
det 2012
det 2013
det 2014
gnb 2009
gnb 2010
gnb 2011
gnb 2012
gnb 2013
gnb 2014
htx 2009
htx 2010
htx 2011
htx 2012
htx 2013
htx 2014
clt 2009
clt 2010
clt 2011
clt 2012
clt 2013
clt 2014
jax 2009
jax 2010
jax 2011
jax 2012
jax 2013
jax 2014
kan 2009
kan 2010
kan 2011
kan 2012
kan 2013
kan 2014
mia 2009
mia 2010
mia 2011
mia 2012
mia 2013
mia 2014
min 2009
min 2010
min 2011
min 2012
min 2013
min 2014
nwe 2009
nwe 2010
nwe 2011
n

In [71]:
# Add in POS
brad_pos = pd.read_csv("brad_positions.csv") #Removed Greg Jones in 2012 from JAX (ILB and FB)

team_dict ={
    'ATL': 'atl','BUF': 'buf','CAR': 'car','CHI': 'chi',
    'CIN': 'cin','CLE': 'cle','IND': 'clt','ARI': 'crd',
    'DAL': 'dal','DEN': 'den','DET': 'det','GB': 'gnb',
    'HOU': 'htx','JAC': 'jax','KC': 'kan','MIA': 'mia',
    'MIN': 'min','NO': 'nor','NE': 'nwe','NYG': 'nyg',
    'NYJ': 'nyj','TEN': 'oti','PHI': 'phi','PIT': 'pit',
    'OAK': 'rai','STL': 'ram','BAL': 'rav','SD': 'sdg',
    'SEA': 'sea','SF': 'sfo','TB': 'tam','WAS': 'was'
}

brad_pos['team'] = brad_pos.apply(lambda row: team_dict[row['team']], axis=1)

df2 = pd.merge(output, brad_pos, how='left', on=['name','year','team'])

bigteam_dict = dict(zip(team_dict.values(),team_dict.keys()))
df2['caps_team'] = df2.apply(lambda row: bigteam_dict[row['team']], axis=1)

df2.head()

,team,year,name,G,Yrs,Age,position,caps_team
0,crd,2009,Hamza Abdullah,1,4,26,CB,ARI
1,crd,2009,Michael Adams,16,2,24,S,ARI
2,crd,2009,Jason Banks,1,Rook,24,NaN,ARI
3,crd,2009,Anthony Becht,16,9,32,TE,ARI
4,crd,2009,Monty Beisel,6,8,31,LB,ARI


In [95]:
# Strip players who didn't play at least 3 games
stripped = pd.read_csv("stripped.csv") #Removed Greg Jones in 2012 from JAX (ILB and FB)

df3 = pd.merge(df2, stripped, how='left', on=['name','year','team'])

df4 = df3[pd.isnull(df3['count'])]

# Limit to relevant columns
df4.loc[:,'team'] = df4.loc[:,'caps_team']
df4 = df4.drop(['count','caps_team'], 1)

# Output
df4.to_csv("roster_info.csv", index = False)

In [118]:
# Strip players in the injury list but not on the roster
roster_nyt = df3[pd.isnull(df3['count'])]
roster_nyt = roster_nyt.loc[:,['name','year','team','G']]

injuries = pd.read_csv("injuries_unique_jkg_clean.csv") #Removed Greg Jones in 2012 from JAX (ILB and FB)
injuries = injuries.drop('Unnamed: 0',1)

injuries_pruned = pd.merge(injuries, roster_nyt, how='inner', on=['name','year','team'])

injuries_pruned.to_csv("injuries_unique_jkg_clean_pruned.csv", index = False)